# Tutorial 1: loading data and running model
This is a tutorial for using the Geomagnetic Field Inversions code written by Frenk Out, Maximilian Schanner, Liz van Grinsven, Monika Korte, and Lennart de Groot. This tutorial will guide you through loading data (1) and running a basic geomagnetic model (2).

### 0. Loading libraries
This tutorial requires, besides geomagnetic_field_inversions, numpy, pandas, and pathlib. The geomagnetic_field_inversions code consists of two modules:
1. InputData is a class that prepares geomagnetic field data to be inputted in FieldInversion later.
2. FieldInversion is the main class where all calculations happen.

In [1]:
import numpy as np
# Necessary for loading excel/csv files and creating DataFrames
import pandas as pd
# Necessary for correct path handling
from pathlib import Path

# Our code
from geomagnetic_field_inversions import InputData, FieldInversion

## 1. Loading data
We will now proceed to loading the data. This is required to run the inversion later. The basic input for adding and fitting the data is a Pandas DataFrame; in the following the name of the column is given between brackets. The DataFrame needs at least a column with latitude (lat; between -90 and 90 degrees), longitude (lon; between 0 and 360 degrees), time (t) of datum, and then data in the form of inclination (I), declination (D), intensity (F), X (X), Y (Y), Z (Z), or H (H) components. Additionally, errors can be appointed to each datatype (dI and dD (or alpha95), dF, dX, dY, dZ, dH).

We employ the data in the `example_data.csv` file.

#### *a) load data*
First we load the file with pandas. After that, we are only using data between -2000 and 1990 AD.

In [2]:
# Set a path to load files
# path should be like: .../geomagnetic_field_inversions/doc
path = Path().absolute()
# load data
dataset = pd.read_csv(path / 'example_data.csv', index_col=0)
# find data corresponding to requested time period
dataset = dataset.query('-2000 <= t <= 1990')
dataset.reset_index(inplace=True, drop=True)
# show data
dataset.head(-10)

,t,lat,lon,F,dF,D,dD,I,dI,colat,rad,dt,FID,UID
0,-2000.0,60.6,23.1,60000.0,5000.0,NaN,NaN,NaN,NaN,29.4,6371.2,100,ERDA/2101/,0
1,-2000.0,36.0,103.8,46500.0,5000.0,NaN,NaN,NaN,NaN,54.0,6371.2,100,ERDA/2101/,1
2,-2000.0,33.0,111.4,45600.0,5000.0,NaN,NaN,NaN,NaN,57.0,6371.2,100,ERDA/2101/,2
3,-2000.0,30.5,111.4,42000.0,5000.0,NaN,NaN,NaN,NaN,59.5,6371.2,100,ERDA/2101/,3
4,-2000.0,-1.9,279.3,38410.0,5000.0,NaN,NaN,NaN,NaN,91.9,6371.2,100,ERDA/2101/,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6262,1980.0,37.5,251.4,NaN,NaN,11.4,5.7,64.2,2.4,52.5,6371.2,100,ERDA/2101/,6262
6263,1980.0,37.5,251.4,NaN,NaN,12.7,5.6,64.1,2.4,52.5,6371.2,100,ERDA/2101/,6263
6264,1980.0,37.5,251.5,NaN,NaN,11.3,5.4,63.1,2.4,52.5,6371.2,100,ERDA/2101/,6264
6265,1980.0,37.5,15.0,43500.0,5000.0,-0.2,3.5,45.6,2.4,52.5,6371.2,100,ERDA/2101/,6265


#### *b) initiate data-class*
with the Pandas DataFrame loaded, we can now initiate an instance of the `InputData`-class, required for using the FieldInversion class later. Arguments for the `InputData`-class are only one Pandas DataFrame. However, you can only include one, so if you have multiple files or DataFrames, you will have to merge them into one!

In [3]:
inputdata = InputData(dataset)

Our data is now ready to be used for the geomagnetic field inversion

## 2. Starting the model
#### *a) FieldInversion*
The `FieldInversion`-class is the 'location' where we will perform all calculations. An instance of the FieldInversion class is called by providing the following arguments:
  1. t_min: start time of model (years)
  2. t_max: end time of model (years)
  3. t_step: modeled time steps (years)
  4. (optional) maxdegree: maximum degree of employed spherical harmonics, defaults to degree 3 (dipole, quadrupole, and octupole components)
  5. (optional) r_model: the radius at which the geomagnetic field is modeled (km), defaults to 6371.2 km
  6. (optional) verbose: verbosity flag - whether the code talks back to you

&rarr; For more info about parameters of this class, type:
`FieldInversion?`

In this tutorial we will use a time array from -2000 to 1990 CE with time steps of 10 year. Furthermore, we will use a spherical model of degree 10 and enable the verbosity flag.

In [4]:
# input modeled time array; from -2000 to 1990 with steps of 10 yr
test_inv = FieldInversion(t_min=-2000, t_max=1990, t_step=200, maxdegree=10, verbose=True)

After we have set up the class, we can proceed to load the `InputData` instance and prepare the matrices for our geomagnetic field inversion.
#### *b) prepare_inversion*
This method is used to add our previously gathered data and determine the spatial and temporal damping type we want to apply.
Damping constrains the Gauss coefficients (model parameters) in space (spatial) and time (temporal). Spatial damping enforces that higher order Gauss coefficients do not become too large, while temporal damping enforces a smoothness condition on the individual Gauss coefficients through time. We have implemented 6 spatial damping types and 2 temporal damping types, which are:
  1. (Spatial): `uniform` -- uniform damping across all coefficients
  2. (Spatial): `ohmic_heating` -- constraining surface heat flux through the core-mantle boundary (CMB)
  3. (Spatial): `energy_diss` -- minimizing effects of advection and diffusion outer core
  4. (Spatial): `powerseries` -- minimal damping of higher degrees
  5. (Spatial): `smooth_core` -- minimizing horizontal derivative of $B_r$ at the CMB
  6. (Spatial): `min_ext_energy` -- minimizing effect crustal fields
  7. (Temporal):`min_vel` -- minimizing $dB_r$/$dt$ over the CMB
  9. (Temporal):`min_acc` -- minimizing $d^2B_r$/$dt^2$ over the CMB

`prepare_inversion` requires the following parameters:
  1. d_inst: instance containing data (`inputdata` in our case)
  2. (optional) spat_type: string indicating the type of spatial damping
     - In this tutorial we use `ohmic_heating` for spatial damping
  4. (optional) temp_type: string indicating the type of temporal damping
     - In this tutorial we use `min_acc` for temporal damping.
  6. (optional) spat_ddip: boolean indicating whether to damp dipole coefficients for spatial damping
     - defaults to False: no damping
  8. (optional) temp_ddip: boolean indicating whether to damp dipole coefficients for temporal damping
     - defaults to True: yes damping

&rarr; For more info on the damping types, please have a look at the accompanying manuscript

In [5]:
test_inv.prepare_inversion(inputdata, spat_type='ohmic_heating', temp_type='min_acc')

Data from t=-2000.0 to t=1990.0
This dataset contains 10044 records from 1725 locations.
It consists of 2827 declinations, 4253 inclinations and 2964 intensities, 0 x-data, 0 y-data, 0 z-data, and 0 h-data.
Calculating Schmidt polynomials and Fréchet coefficients
Calculating spatial damping matrix
Calculating temporal damping matrix
Calculations finished


With the verbosity flag enabled `prepare_inversion` outputs info on the dataset: we mainly have inclination data, besides declination and intensity data. Now we can run the model.

#### *c) run_inversion*
We now proceed to starting the iterative inversion by using the `run_inversion`-method. This method requires:
  1. x0: a starting model with length equal to the number of Gauss coefficients
     - The correct length can always be invoked by using `test_inv._nr_coeffs`
     - We will use a starting model with where only $g_1^0$ (first coefficient) has a value of -30000
  2. spat_damp: spatial damping factor. If no damping required, set to zero
     - For our spatial damping we use a value of 1e-13
  3. temp_damp: temporal damping factor. If no damping required, set to zero
     - For our temporal damping we use a value of 1e-3
  5. (optional) max_iter: the maximum number of iterations
     - Defaults to 10 iterations
  6. (optional) stop_crit: stopping criterion. if the change in residual is smaller than this value, the iterations are stopped.
  7. (optional) path: pathlib.Path object that allows for saving matrices, allowing (intense) calculation of standard deviations and full covariance matrices (see calc_stdev in tools/core.py).

Normally, you would find the optimal damping parameters by sweeping through a range of damping parameters and find a combination that both minimizes the residual and the damping energy (dependent on Gauss coefficient). See Tutorial 3 for more about that.

In [6]:
# our starting model should be as long as the number of gaussian coefficients up to degree 10, i.e. 120 coefficients.
x0 = np.zeros(120)
# you could also make use of the attribute _nr_coeffs to do this automatically:
# x0 = np.zeros(test_inv._nr_coeffs)
x0[0] = -30000
test_inv.run_inversion(x0, max_iter=5, spat_damp=1.0e-13, temp_damp=1.0e-3)

Setting up starting model
Start calculations iteration 0
Residual is 1.97
Prepare and solve equations
Start calculations iteration 1
Residual is 1.18
Prepare and solve equations
Start calculations iteration 2
Residual is 1.14
Prepare and solve equations
Start calculations iteration 3
Residual is 1.14
Prepare and solve equations
Start calculations iteration 4
Residual is 1.14
Prepare and solve equations
Calculating optional spatial and temporal norms
Spatial damping norm: 1.66e+12
Temporal damping norm: 8.88e+01
Finished inversion


After each iteration, the rms residual is shown and how many datapoins are rejected. After 3 iterations the residual does not change much.

#### *d) save_spherical_coefficients*
Finally we will save our final coefficients by inputting a path and name to save our files; we save the residuals by setting `save_residual=True`. This will create a *Tutorial_residual.csv*-file containing all residuals after each timestep. Additionally, we save the final time-dependent Gauss coefficients of the final iteration, per timestep. The gaussian coefficients are stored per row degree-wise, so: $g_1^0$, $g_1^1$, $h_1^1$, $g_2^0$, $g_2^1$, $h_2^1$, $g_2^2$, $h_2^2$, etc ...

In [7]:
test_inv.save_coefficients(path / 'output', file_name='Tutorial1', save_residual=True)

Besides saving the coefficients in this format, you can also save the coefficient in a fortran format using the `save_to_fortran_format`-method by providing only the name of the to be created file (Path-object or string). Furthermore, you can save your created model by using the `result_to_pymagglobal`-method by providing a name (string) for the model. The outputted model is suitable for use in (plotting tools of) pymagglobal.

##### Note on sampling Gauss coefficients
The `FieldInversion`-class contains two methods (`sample_prior` and `sample_posterior`, resp.) for sampling Gauss coefficients (default 100) from the prior and posterior distribution. Both methods only require, in order, the spatial and temporal damping factor. Both methods output time-independent Gauss coefficients sorted per spline, per degree and order, and then per individual sample. See Tutorial_2 for an example of their usage

## End of tutorial
This is the end of the tutorial concerning basic loading of data and running of model.

&rarr; Tutorial_2 covers plotting of results and some small functionalities

&rarr; Tutorial_3 covers sweeping through damping parameters to find the best model

&rarr; Tutorial_4 covers reading geomagia data

If you want to know more about a function, have a look at the code, or type the function with a questionmark after it to get more info (e.g. `FieldInversion.sample_prior?`). Otherwise you can allways drop an email to Frenk Out at f.out@uu.nl or outfrenk@gmail.com.